# フーリエ変換

### 時間領域から周波数領域へ

私たちが普段聞いている音は、空気の圧力変動が時間の経過と共にどう変化するかを示した**波形（時間領域の信号）**です。しかし、この波形を見ただけでは、音の高さ（周波数）や音色を理解するのは困難です。

フーリエ変換は、この「時間領域」の信号を、「どの周波数の成分がどれくらいの強さで含まれているか」を示す**スペクトル（周波数領域の信号）**に変換する強力な数学的ツールです。これにより、音の特性を分析できます。

- __基本周波数__: スペクトルの中で最も低い、強いピーク。音の基本的な高さを決定します。
- __倍音__: 基本周波数の整数倍の周波数を持つ成分。倍音の構成バランスが、ピアノやギターといった楽器の音色の違いを生み出します。

### 準備：ライブラリのインポート

音声ファイルを扱うためのライブラリを追加でインポートします。

- requests: インターネット上からファイルをダウンロードします。
- scipy.io.wavfile: WAV形式の音声ファイルを読み書きします。
- scipy.fft: 高速フーリエ変換（FFT）および逆変換（IFFT）を実行します。

In [ ]:
import urllib.request
from scipy.io import wavfile
from scipy import fft
import numpy as np
import matplotlib.pyplot as plt

### 音声ファイルの読み込みと波形のプロット

ダウンロードしたWAVファイルを読み込み、元のRコードと同様に、波形の一部（0.5秒から0.7秒）を切り出してプロットします。

ファイルは https://freewavesamples.com にあるものを使います[（ライセンス情報）](https://freewavesamples.com/about-us-license)。

In [ ]:
# フリーWAVサンプルのURL
url_wav = 'https://freewavesamples.com/files/Alesis-Fusion-Steel-String-Guitar-C4.wav'
file_name = 'guitar_sound.wav'

# ファイルをダウンロード
print(f"Downloading {file_name} from {url_wav}...")
try:
    urllib.request.urlretrieve(url_wav, file_name)
    print("Download complete.")
except Exception as e:
    print(f"An error occurred during download: {e}")


In [ ]:
# WAVファイルを読み込む
sampling_rate, data = wavfile.read(file_name)

# ステレオかモノラルかを確認し、モノラルに変換
if data.ndim > 1:
    # 左右の平均をとってモノラル化
    audio_data = data.mean(axis=1)
else:
    audio_data = data

print(f"Sampling Rate: {sampling_rate} Hz")
print(f"Data Shape: {audio_data.shape}")
print(f"Duration: {len(audio_data) / sampling_rate:.2f} seconds")


# 対象とする開始時間と終了時間を設定
start_time = 0.5  # seconds
end_time = 0.51   # seconds

# サンプルインデックスに変換
start_sample = int(start_time * sampling_rate)
end_sample = int(end_time * sampling_rate)

# データを切り出す
clipped_data = audio_data[start_sample:end_sample]
time_axis = np.linspace(start_time, end_time, num=len(clipped_data))

# 音の波形（時間関数）を表示
plt.figure(figsize=(12, 5))
plt.plot(time_axis, clipped_data)
plt.title('Audio Waveform (Time Domain)')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.show()

書き込んでください
- 横軸：
- 縦軸：

2.5. フーリエ変換と振幅スペクトルの表示

切り出した音声データに**高速フーリエ変換（FFT）**を適用し、周波数スペクトルを計算します。横軸が周波数(Hz)、縦軸がその周波数成分の振幅（強さ）を示すグラフになります。

In [ ]:
# 高速フーリエ変換（FFT）を実行
N = len(clipped_data)
yf = fft.fft(clipped_data)
# 周波数軸を生成
xf = fft.fftfreq(N, 1 / sampling_rate)

# FFTの結果は複素数なので絶対値をとって振幅を求める
# 結果は対称なので、正の周波数成分のみ（前半部分）をプロットする
positive_freq_indices = np.where(xf >= 0)
xf_pos = xf[positive_freq_indices]
yf_pos_amp = np.abs(yf[positive_freq_indices])

# 振幅スペクトルの表示
plt.figure(figsize=(12, 5))
# type="h" のような棒グラフで表示
plt.vlines(xf_pos, ymin=0, ymax=yf_pos_amp, color='b')
plt.title('Amplitude Spectrum (Frequency Domain)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim(0, 4000) # 見やすくするため、4000Hzまでの範囲を表示
plt.show()

__グラフの読み方__： このグラフに現れるいくつかの大きなピークが、このピアノの音を構成している主要な周波数です。最も左側にある大きなピークが基本周波数で、音の「ド」の高さを決めています。それより右側にあるピークは倍音で、このピアノ独特の豊かな音色を作り出しています。

書き込んでください
- 横軸：
- 縦軸：

### 逆フーリエ変換による波形の復元

最後に、周波数スペクトルから**逆高速フーリエ変換（IFFT）**を行うことで、元の時間領域の波形を完全に復元できることを確認します。これは、フーリエ変換が情報を失うことなく、単に信号の表現形式を変えているだけであることを示しています。

In [ ]:
# 逆高速フーリエ変換（IFFT）を実行
restored_data = fft.ifft(yf)

# IFFTの結果は複素数なので実部を取り出す
restored_data_real = np.real(restored_data)

# 逆フーリエ変換の結果を表示
plt.figure(figsize=(12, 5))
plt.plot(time_axis, restored_data_real, color='green')
plt.title('Restored Audio Waveform via IFFT')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
# 元の波形と同じスケールに合わせる
plt.ylim(np.min(clipped_data), np.max(clipped_data))
plt.show()

書き込んでください
- 横軸：
- 縦軸：

### 📖 課題

別のファイル https://freewavesamples.com にある別のファイルを用いて、上記の処理を実行してください。

解析する開始時間や終了時間は適宜変更してください。

### 📖 総括課題

今回扱った

- フーリエ係数
- フーリエ級数展開
- フーリエ変換
- 逆フーリエ変換

の 4 項目について、自分の言葉でまとめてください。以下の言葉を含めてください。

- 近似
- 周期 2$\pi$
- 時間
- 周波数

（総括課題の回答欄）